In [1]:
#@title <h1>Step1: Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install ffmpeg-python

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 417 (delta 0), reused 2 (delta 0), pack-reused 414
Receiving objects: 100% (417/417), 548.34 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (229/229), done.
--2023-07-21 15:15:30--  https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435801865 (416M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip/checkpoints/wav2lip_gan.pth’

/content/Wav2Lip/ch 100%[===================>] 415.61M  36.5MB/s    in 14s     

2023-07-21 15:15:45 (29.5 MB/

In [23]:
!pip install pytube
from pytube import YouTube

start_time = 0  # Start time in seconds
end_time = 15    # End time in seconds

video_url = 'https://www.youtube.com/watch?v=YMuuEv37s0o'
yt = YouTube(video_url)

video = yt.streams.get_highest_resolution()
video_file = 'video.mp4'
video.download(filename=video_file)

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

output_file = 'trimmed_video.mp4'
ffmpeg_extract_subclip(video_file, start_time, end_time, targetname=output_file)

import os
os.remove(video_file)




Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [24]:
# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/trimmed_video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

In [4]:
#import audio from local drive

from google.colab import files
uploaded = files.upload()

Saving output10.wav to output10.wav


In [26]:
!pip install librosa
import librosa
import soundfile as sf
# Load the audio file
file_path = '/content/audio.wav'
wav, sample_rate = librosa.load(file_path, sr=16000)

# Perform cropping (adjust the start and end times accordingly)
start_time = 0.0  # Crop from 2 seconds
end_time = 15.0   # Crop until 10 seconds
start_index = int(start_time * sample_rate)
end_index = int(end_time * sample_rate)
cropped_wav = wav[start_index:end_index]

output_path = '/content/cropped_audio.wav'
sf.write(output_path, cropped_wav, sample_rate)

In [27]:
#@title STEP4: Start Crunching and Preview Output
#@markdown <b>Note: Only change these, if you have to</b>
pad_top =  10#@param {type:"integer"}
pad_bottom =  10#@param {type:"integer"}
pad_left =  5#@param {type:"integer"}
pad_right =  5#@param {type:"integer"}
rescaleFactor =  1#@param {type:"integer"}
nosmooth = False #@param {type:"boolean"}

if nosmooth == False:
  !cd Wav2Lip && python inference.py --checkpoint_path /content/Wav2Lip/checkpoints/wav2lip_gan.pth --face "/content/input_video.mp4" --audio "/content/cropped_audio.wav" --outfile "/content/sample_data/output.mp4" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !cd Wav2Lip && python inference.py --checkpoint_path /content/Wav2Lip/checkpoints/wav2lip_gan.pth --face "/content/input_video.mp4" --audio "/content/cropped_audio.wav" --outfile "/content/sample_data/output.mp4" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 376
(80, 1201)
Length of mel chunks: 372
  0% 0/3 [00:00<?, ?it/s]
  0% 0/24 [00:00<?, ?it/s]
  4% 1/24 [03:46<1:26:45, 226.31s/it]
  8% 2/24 [07:40<1:24:44, 231.11s/it]
 12% 3/24 [11:25<1:19:47, 228.00s/it]
 17% 4/24 [15:11<1:15:49, 227.49s/it]
 21% 5/24 [18:57<1:11:48, 226.74s/it]
 25% 6/24 [22:51<1:08:45, 229.18s/it]
 29% 7/24 [26:39<1:04:50, 228.84s/it]
 33% 8/24 [30:27<1:00:58, 228.69s/it]
 38% 9/24 [34:16<57:13, 228.88s/it]  
 42% 10/24 [38:00<53:00, 227.18s/it]
 46% 11/24 [41:51<49:28, 228.36s/it]
 50% 12/24 [45:40<45:41, 228.50s/it]
 54% 13/24 [49:31<42:03, 229.38s/it]
 58% 14/24 [53:16<38:01, 228.18s/it]
 62% 15/24 [57:04<34:13, 228.12s/it]
 67% 16/24 [1:00:51<30:21, 227.63s/it]
 71% 17/24 [1:04:35<26:25, 226.48s/it]
 75% 18/24 [1:08:20<22:37, 226.17s/it]
 79% 19/24 [1:12:03<18:45, 225.11s/it]
 83% 20/24 [1:15:46<14:58, 224.68s/it]
 88% 21/24 [1:19:31<11:14, 224.77s/it]
 92% 22/24 [1:23:

In [14]:
from google.colab import files
files.download('/content/sample_data/output.mp4')
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/sample_data/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>